<a href="https://colab.research.google.com/github/pufeiyang/Deep_Learning_Regression/blob/main/ResNet_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Modules

In [1]:
from tensorflow.keras import layers,models
from tensorflow.keras import callbacks
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from sklearn.metrics import mean_squared_error

# The identity block is the block that has no conv layer at shortcut.
## Arguments
input_tensor: input tensor  
units:output shape
## Returns
Output tensor for the block.

In [2]:
def identity_block(input_tensor,units):

	x = layers.Dense(units)(input_tensor)
	x = layers.BatchNormalization()(x)
	x = layers.Activation('relu')(x)

	x = layers.Dense(units)(x)
	x = layers.BatchNormalization()(x)
	x = layers.Activation('relu')(x)

	x = layers.Dense(units)(x)
	x = layers.BatchNormalization()(x)

	x = layers.add([x, input_tensor])
	x = layers.Activation('relu')(x)

	return x

# A block that has a dense layer at shortcut.
## Arguments
input_tensor: input tensor  
unit: output tensor shape
## Returns
Output tensor for the block

In [3]:
def dens_block(input_tensor,units):

	x = layers.Dense(units)(input_tensor)
	x = layers.BatchNormalization()(x)
	x = layers.Activation('relu')(x)

	x = layers.Dense(units)(x)
	x = layers.BatchNormalization()(x)
	x = layers.Activation('relu')(x)

	x = layers.Dense(units)(x)
	x = layers.BatchNormalization()(x)

	shortcut = layers.Dense(units)(input_tensor)
	shortcut = layers.BatchNormalization()(shortcut)

	x = layers.add([x, shortcut])
	x = layers.Activation('relu')(x)
	return x

# Instantiates the ResNet50 architecture.
## Arguments        
input_tensor: optional Keras tensor (i.e. output of `layers.Input()`) to use as input for the model.        
## Returns
A Keras model instance.

In [4]:
def ResNet50Regression():
	
	Res_input = layers.Input(shape=(141,))

	width = 16

	x = dens_block(Res_input,width)
	x = identity_block(x,width)
	x = identity_block(x,width)

	x = dens_block(x,width)
	x = identity_block(x,width)
	x = identity_block(x,width)
	
	x = dens_block(x,width)
	x = identity_block(x,width)
	x = identity_block(x,width)

	x = layers.BatchNormalization()(x)
	x = layers.Dense(1, activation='linear')(x)
	model = models.Model(inputs=Res_input, outputs=x)

	return model

# Prepare Dataset
## Read Dataset
Shape of Dataset:(336,145) 
## Preprocess Dataset
Normalization and Standardization. 
## Split Dataset
Shape of x_train:(252, 141)  
Shape of y_train:(252, 1)  
Shape of x_test:(84, 141)  
Shape of y_test:(84, 1)

In [5]:
plt.switch_backend('agg')

path = "/content/dataset.xlsx"
dataSet = pd.read_excel(path)

dataSet = np.array(dataSet)
# print(dataSet.shape)

x = dataSet[:,3:-1]
y = dataSet[:,2]
y = y.reshape(-1,1)
# print(x.shape)
# print(y.shape)

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_x.fit(x)
xscale = scaler_x.transform(x)
scaler_y.fit(y)
yscale = scaler_y.transform(y)
X_train, X_test, y_train, y_test = train_test_split(xscale, yscale,test_size=0.25)
# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)

FileNotFoundError: ignored

# Build Model

In [51]:
model = ResNet50Regression()

model.compile(loss='mse', optimizer='adam', metrics=['mse'])
# model.summary()

#compute running time
starttime = datetime.datetime.now()

history = model.fit(X_train, y_train, epochs=20, batch_size=5000, verbose=2, callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=10,verbose=2, mode='auto')], validation_split=0.1)
#history = model.fit(X_train, y_train, epochs=10, batch_size=60000,  verbose=1, validation_split=0.1)
endtime = datetime.datetime.now()

Epoch 1/20
1/1 - 10s - loss: 3.2823 - mse: 3.2823 - val_loss: 0.3966 - val_mse: 0.3966 - 10s/epoch - 10s/step
Epoch 2/20
1/1 - 0s - loss: 2.5212 - mse: 2.5212 - val_loss: 0.3620 - val_mse: 0.3620 - 39ms/epoch - 39ms/step
Epoch 3/20
1/1 - 0s - loss: 2.1191 - mse: 2.1191 - val_loss: 0.3429 - val_mse: 0.3429 - 48ms/epoch - 48ms/step
Epoch 4/20
1/1 - 0s - loss: 1.7480 - mse: 1.7480 - val_loss: 0.3344 - val_mse: 0.3344 - 42ms/epoch - 42ms/step
Epoch 5/20
1/1 - 0s - loss: 1.5296 - mse: 1.5296 - val_loss: 0.3294 - val_mse: 0.3294 - 39ms/epoch - 39ms/step
Epoch 6/20
1/1 - 0s - loss: 1.3341 - mse: 1.3341 - val_loss: 0.3280 - val_mse: 0.3280 - 48ms/epoch - 48ms/step
Epoch 7/20
1/1 - 0s - loss: 1.2287 - mse: 1.2287 - val_loss: 0.3290 - val_mse: 0.3290 - 41ms/epoch - 41ms/step
Epoch 8/20
1/1 - 0s - loss: 1.0885 - mse: 1.0885 - val_loss: 0.3304 - val_mse: 0.3304 - 39ms/epoch - 39ms/step
Epoch 9/20
1/1 - 0s - loss: 0.9949 - mse: 0.9949 - val_loss: 0.3318 - val_mse: 0.3318 - 41ms/epoch - 41ms/step
Ep

# Save Model

In [53]:
model.save('/content/My_ResnetModel.h5')
# plot_model(model, to_file='/content/ResnetModel.png')

#from keras.models import load_model
#model = load_model('/content/My_ResnetModel.h5') 

# Model Predicting

In [62]:
yhat = model.predict(X_test)

print('The time cost: ')
print(endtime - starttime)
print('The test loss: ')
print(mean_squared_error(yhat,y_test))

#invert normalize
yhat = scaler_y.inverse_transform(yhat) 
y_test = scaler_y.inverse_transform(y_test)

The time cost: 
0:00:10.561498
The test loss: 
1.4293041311959368e+36


# Visualize Model

In [55]:
# "Loss"
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
# plt.show()
plt.savefig('/content/OptimalModelDataSet.png')

plt.figure()
plt.plot(y_test[0:4000],'rx')
plt.plot(yhat[0:4000],' go',markerfacecolor='none')
plt.title('Result for ResNet Regression')
plt.ylabel('Y value')
plt.xlabel('Instance')
plt.legend(['Real value', 'Predicted Value'], loc='upper right')
plt.savefig('/content/OptimalModelDataSet2.png')
# plt.show()

file = open('/content/dataset.txt','r+')
file.write('predicted ' + 'observed ' + '\n')
for i in range(len(yhat)):
	file.write(str(yhat[i][0])+' '+str(y_test[i][0])+'\n')
file.close()